In [29]:
import psycopg2
from keys import secrets
import csv
import pandas as pd
import sqlalchemy

In [23]:
connection = psycopg2.connect(
    host = 'steam-db.c5m99euxia00.us-east-1.rds.amazonaws.com',
    user = secrets.get('user'),
    password = secrets.get('password'),
    database='steam_db'
    )
cursor=connection.cursor()

In [24]:
# Create steam tags table
cursor.execute ("""CREATE TABLE tags(
id SERIAL PRIMARY KEY,
genre text
)""")

# # Create steam games table
# cursor.execute("""CREATE TABLE steam(
# name text,
# tags int,
# windows text,
# mac text,
# linux text,
# reviews text,
# release text,
# discount text,
# price double precision
# )""")

connection.commit()

In [30]:
# Gather all available tables and columns info

sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

C:\Users\admin\AppData\Local\Temp\ipykernel_34344\1120550366.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, con=connection)


,table_name,column_name,data_type,table_schema
0,tags,id,integer,public
1,tags,genre,text,public


In [32]:
# Use tags to create table
with open('data/tags.csv', 'r') as row:
    cursor.copy_from(row, 'tags', sep=',')

# # Use steam to create table
# with open('data/steam.csv', 'r') as row:
#     cursor.copy_from(row, 'steam', sep=',')

connection.commit()

# Create a new role for additional access
cursor.execute("""CREATE ROLE guest WITH LOGIN PASSWORD 'guest_access'""")

# Grant different privileges
cursor.execute("""GRANT CONNECT ON DATABASE steam_db TO guest""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON tags TO guest""")

In [39]:
# Read from table tags

sql = """
SELECT * FROM tags
"""

pd.read_sql(sql, con=connection)

C:\Users\admin\AppData\Local\Temp\ipykernel_34344\3703861811.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, con=connection)


,id,genre
0,492,Indie
1,1003823,Profile Features Limited
2,19,Action
3,597,Casual
4,4182,Singleplayer
...,...,...
439,363767,Snooker
440,42329,Coding
441,30927,Fox
442,1220528,Hobby Sim
